## To do Edson speed limit: 
- Import Aimsun shapefile
- Change the speed of off-ramp
- From [Google Maps API](https://developers.google.com/maps/documentation/roads/speed-limits), get speed limit on major roads:
    - Major road in Fremont (How to select them in Aimsun: use their name in Aimsun)
    - Highway and on/off-ramp (How to select them in Aimsun: road with speed limit >= 100 km/h)
- save the new roads somewhere as a shapefile
- (later) import the shapefile inside Aimsun ([see link](https://www.aimsun.com/tech_notes/gis-import-1/))

## To do Edson external TAZs:
- create a external demand delimitation:
    - load SFCTA data as Geopandas point (one point = one origin or one destination)
    - Get convex hull of the point
    - Use the convex hull (+ buffer) as the external demand delimation
- create external centroids:
    - select road with no fnode and capacity above 800
    - create a point at the end of all selected road
    - plot the points, get a list of points to remove visualy
- create external TAZs:
    - create a mesh a points inside the external demand delimitation and outside the internal demand delimitation
    - use a Direction API (maybe Here direction):
        - for every mesh point:
            - Query path from mesh point to center of the project area
            - Find the closest external centroid to the path. Test that all path are not to far from existing external centroid --> if not, we might be missing one external centroid.
            - Associate the external centroid to the mesh point.
        - create external TAZ from mesh of points (if you reach point, Theo has already done it for internal TAZs)

In [ ]:
## Ignore the following

In [1]:
# --- Global variables

# Setting up the Coordinate Reference Systems up front in the necessary format.
crs_degree = {'init': 'epsg:4326'} # CGS_WGS_1984 (what the GPS uses)

# --- Paths

# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
# Root path of the Dropbox business account
dbx = get_dropbox_location()

# Temporary! Location of the folder where the restructuring is currently happening
data_path = dbx + '/Private Structured data collection'

In [2]:
import geopandas as gpd
from shapely import geometry

## 1. Creation of the project delimitation

In [3]:
## create the project delimitation

# this code creates the project delimitation from a set of points
p = []
p.append((-121.94277062699996, 37.55273259000006))
p.append((-121.94099807399999, 37.554268507000074))
p.append((-121.91790942699998, 37.549823434000075))
p.append((-121.89348666299998, 37.52770136500004))
p.append((-121.90056572499998, 37.52292299800007))
p.append((-121.90817571699995, 37.52416183400004))
p.append((-121.91252749099999, 37.51845069500007))
p.append((-121.91349347899995, 37.513972023000065))
p.append((-121.90855417099999, 37.503837324000074))
p.append((-121.91358547299996, 37.50097863000008))
p.append((-121.90798018999999, 37.49080413200005))
p.append((-121.91894942199997, 37.48791568200005))
p.append((-121.92029048799998, 37.488706567000065))
p.append((-121.93070953799997, 37.48509600500006))
p.append((-121.93254686299997, 37.48864173700008))
p.append((-121.94079404499996, 37.50416395900004))
p.append((-121.94569804899999, 37.51332606200003))
p.append((-121.94918207899997, 37.520371545000046))
p.append((-121.95305006999996, 37.52804520800004))
p.append((-121.953966735, 37.53272020000003))
p.append((-121.95428756799998, 37.53817435800005))
p.append((-121.95506236799997, 37.54107322100003))
p.append((-121.95676186899999, 37.54656695700004))
p.append((-121.95529950799994, 37.54980786700003))
p.append((-121.95261192399994, 37.550479763000055))
p.append((-121.94988481799999, 37.55277211300006))
p.append((-121.94613010599994, 37.55466923100005))
p.append((-121.94277062699996, 37.55273259000006))

delimitation_poly = geometry.Polygon([[po[0], po[1]] for po in p])

project_del = gpd.GeoDataFrame({'Type': ['Delimitation','Box'], 'geometry': [delimitation_poly, delimitation_poly.envelope]})

lat = [i[0] for i in delimitation_poly.envelope.exterior.coords]
lng = [i[1] for i in delimitation_poly.envelope.exterior.coords]
lat_min = min(lat)
lat_max = max(lat)
lng_min = min(lng)
lng_max = max(lng)
print("Box delimitation: " + str((lat_min, lat_max, lng_min, lng_max)))

project_del.crs = 'epsg:4326'
project_del_path = dbx + "/Private Structured data collection/Manual-made dataset (do not touch)/Network/Map/Project Delimitation/Project_delimitation.shp"
project_del.to_file(project_del_path)


Box delimitation: (-121.95676186899999, -121.89348666299998, 37.48509600500006, 37.55466923100005)


## download the OSM network

- See https://github.com/jwass/geopandas_osm
- See http://overpass-turbo.eu
- See https://automating-gis-processes.github.io/CSC18/course-info/Installing_Anacondas_GIS.html
- See https://github.com/gboeing/osmnx-examples/blob/master/notebooks/03-example-osm-place-network.ipynb
- See https://automating-gis-processes.github.io/CSC18/lessons/L3/retrieve-osm-data.html
- Maybe this doc is relevant https://doc.arcgis.com/en/cityengine/2019.0/help/help-import-osm.htm


### Add download of public traces + historical network data

### Parse OSM network to check:
- if roads have speed limits, capacity, and cost functions
- if they are traffic lights and stop signs information
- if they are points of interest
- turn restrictions
- historical data (when the road sections has been created)

### Then convert OSM to shapefile and render it with Kepler.gl

## To do: From OSM data create plots like the following ones (by parsing the data and converting it to shapefile):

In [10]:
import ogr
from shapely.geometry import Point

driver=ogr.GetDriverByName('OSM')
path_osm = data_path + "/Raw data (do not touch)/Network/Map/OSM/"


data = driver.Open(path_osm + 'map1111.osm')

for i in range(data.GetLayerCount()):
    print(data.GetLayerByIndex(i).GetName())
    
layer = data.GetLayer('points')

features=[x for x in layer]
print(len(features))

data_list=[]
i = 0
for feature in features:
    i = i + 1
    data=feature.ExportToJson(as_object=True)
    coords=data['geometry']['coordinates']
    shapely_geo=Point(coords[0],coords[1])
    name=data['properties']['name']
    highway=data['properties']['highway']
    print(highway)
    other_tags=data['properties']['other_tags']
    if other_tags and 'amenity' in other_tags:
        feat=[x for x in other_tags.split(',') if 'amenity' in x][0]
        amenity=feat[feat.rfind('>')+2:feat.rfind('"')]
    else:
        amenity=None
    data_list.append([name,highway,amenity,shapely_geo])
    if i > 20:
        break
# gdf=gpd.GeoDataFrame(data_list,columns=['Name','Highway','Amenity','geometry'],crs={'init': 'epsg:4326'}).to_crs(epsg=3310)


points
lines
multilinestrings
multipolygons
other_relations
1885
motorway_junction
motorway_junction
crossing
None
None
None
None
None
None
None
None
None
None
motorway_junction
traffic_signals
turning_circle
traffic_signals
motorway_junction
turning_circle
turning_circle
turning_circle


In [4]:
from keplergl import KeplerGl
import geopandas as gpd
import pandas as pd
import numpy as np
# --- Global variables

# Setting up the Coordinate Reference Systems up front in the necessary format.
crs_degree = {'init': 'epsg:4326'} # CGS_WGS_1984 (what the GPS uses)

# --- Paths

# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
# Root path of the Dropbox business account
dbx = get_dropbox_location()

# Temporary! Location of the folder where the restructuring is currently happening
data_path = dbx + '/Private Structured data collection'

def to_gdf(path):
    gdf = gpd.GeoDataFrame.from_file(path)
    gdf = gdf.to_crs('epsg:4326')
    return gdf

aimsun_path = dbx + '/Private Structured data collection/Aimsun/Inputs/'

nodes = to_gdf(aimsun_path +'nodes.shp')
sections = to_gdf(aimsun_path +'sections.shp')

fremont_map = KeplerGl(height=600)
fremont_map.add_data(data = nodes, name="nodes")
fremont_map.add_data(data = sections, name="sections")
fremont_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'nodes': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

In [5]:
def to_gdf_csv(path):
# https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html#from-wkt-format
    df = pd.read_csv(path)
    gdf = gpd.GeoDataFrame(
        df, crs='epsg:4326', geometry=gpd.points_from_xy(df.x, df.y))
    return gdf

network_infra_path = data_path + "/Manual-made dataset (do not touch)/Network/Infrastructure/"

stop_signs = to_gdf_csv(network_infra_path + "Stop signs location/Stop_Signs.csv")
traffic_lights = to_gdf_csv(network_infra_path + "Traffic lights location/Traffic_Lights.csv")

fremont_map = KeplerGl(height=600)
fremont_map.add_data(data = stop_signs, name="Stop signs")
fremont_map.add_data(data = traffic_lights, name="Traffic lights")
fremont_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Stop signs': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, …

In [6]:
POIs = pd.read_csv(data_path + "/Data processing/Raw/Network/KPIs/locations_crawl.csv")
# Creating a Geographic data frame for Point of Interests
POIs_gdf = gpd.GeoDataFrame(POIs, crs=crs_degree, geometry=gpd.points_from_xy(POIs.Long, POIs.Lat))


intersections = gpd.overlay(POIs_gdf, project_del[project_del.Type == "Delimitation"], how='intersection')

map_2 = KeplerGl(height=1000)
map_2.add_data(data=intersections, name = "Point Of Interests")
map_2

/Users/theophile/miniconda3/envs/dev/lib/python3.7/site-packages/geopandas/base.py:48: UserWarning: GeoSeries crs mismatch: +init=epsg:4326 +type=crs and epsg:4326
  warn("GeoSeries crs mismatch: {0} and {1}".format(this.crs, other.crs))


User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Point Of Interests': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, …

In [ ]:
## external centroids with OSM network and closer delimitation

In [ ]:
## Create the external TAZs using Google Maps API

In [ ]:
## Get google maps POIs

In [ ]:
## Get neighborhood information + other shapefile data

In [ ]:
## Do k-mean clustering to create internal centroids

In [ ]:
## Get boundaries of internal TAZs

In [ ]:
## Create TAZs

In [ ]:
## Create centroid connections

In [ ]:
## Create files for Aimsun with centroid connections and centroid and OSM network

See [Aimsun documentation to create the layers](https://www.aimsun.com/tech_notes/gis-import-1/)

In [ ]:
## Download Google Maps tiles corresponding to the project